### Notebook to convert sea ice conentration data from NSIDC to lat/lon coords

So far, this only works for v3 data (which contain lat/lon coordinates, so the project conversion is easy). V4 data ([link](https://polarwatch.noaa.gov/catalog/ice-sq-nh-nsidc-cdr-v4/preview/?dataset=daily&var=cdr_seaice_conc&time_min=2021-05-31T00:00:00Z&time_max=2021-05-31T00:00:00Z&proj=epsg3413&colorBar=KT_ice|||0|1|)) don't have lat/lon so it's messier.

Uses nctoolkit

k.drushka // jan 2022

In [1]:
import os
import glob
import nctoolkit as nc

nctoolkit is using Climate Data Operators version 1.9.9


Data product: daily, 25km, v4 product SIC from NSIDC, downloaded from sidads.colorado.edu 

Script to grab the Aug & Sept data from 2014-2020 via ftp using wget (stores to directory ./sic_daily_v3/)



    for y in {2014..2020}
    do
            for m in {08..09}
            do
                    for d in {01..31}
                        do wget -nc  ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02202_V3/north/daily/${y}/seaice_conc_daily_nh_f17_${y}${m}${d}_v03r01.nc
                    done
            done
    done






NOTE: this is the script to grab the v4 data: (stores to directory ./sic_daily_v4/)

    for y in {2014..2020}
    do
        for m in {08..09}
        do
            for d in {01..31} 
            do wget -nc -P sic_daily_v4  ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02202_V4/north/daily/${y}/seaice_conc_daily_nh_${y}${m}${d}_f17_v04r00.nc
            done
        done
    done


In [2]:
# directory where data files are stored
version = 3 # must be 3.... version 4 not yet implemented
ddir = f'/data1/sassie/satellite/seaice/sic_daily_v{version}/'

# make "regridded" file for outputs, if it doesn't exist
savedir = ddir + 'regridded/'
if not(os.path.isdir(savedir)):
    os.mkdir(savedir)
print(f'saving output to {savedir}')

saving output to /data1/sassie/satellite/seaice/sic_daily_v3/regridded/


In [ ]:
# list of files
fg = sorted(glob.glob(ddir + '**nc'))
# files to go through (all)
fis = range(len(fg))

# loop through files
for fi in fis:
    # --- select data: all lons, lats 30 to 90 ---
    thisf=fg[fi]
    # output filename: "regridded" folder, insert "regridded" into the filename
    fnout = thisf.replace('seaice_conc_daily_nh_' , 'regridded/seaice_conc_daily_nh_regridded_')
    # check if regridded (output) file  exists 
    if not(os.path.isfile(fnout)):
        print(thisf , '(' , fi, 'of', len(fis), ')')
        data = nc.open_data(thisf)
        if version == 3:
            # convert to lat/lon
            data.to_latlon(lon=[0, 360], lat=[30, 90], res=0.1)
        elif version == 4:
            print('version 4 not yet implemented - quitting')
            break
        # save output to fnout    
        data.to_nc(fnout)

/data1/sassie/satellite/seaice/sic_daily_v3/seaice_conc_daily_nh_f17_20120801_v03r01.nc ( 0 of 672 )
/data1/sassie/satellite/seaice/sic_daily_v3/seaice_conc_daily_nh_f17_20120802_v03r01.nc ( 1 of 672 )
/data1/sassie/satellite/seaice/sic_daily_v3/seaice_conc_daily_nh_f17_20120803_v03r01.nc ( 2 of 672 )
/data1/sassie/satellite/seaice/sic_daily_v3/seaice_conc_daily_nh_f17_20120804_v03r01.nc ( 3 of 672 )
/data1/sassie/satellite/seaice/sic_daily_v3/seaice_conc_daily_nh_f17_20120805_v03r01.nc ( 4 of 672 )
/data1/sassie/satellite/seaice/sic_daily_v3/seaice_conc_daily_nh_f17_20120806_v03r01.nc ( 5 of 672 )
/data1/sassie/satellite/seaice/sic_daily_v3/seaice_conc_daily_nh_f17_20120807_v03r01.nc ( 6 of 672 )
/data1/sassie/satellite/seaice/sic_daily_v3/seaice_conc_daily_nh_f17_20120808_v03r01.nc ( 7 of 672 )
/data1/sassie/satellite/seaice/sic_daily_v3/seaice_conc_daily_nh_f17_20120809_v03r01.nc ( 8 of 672 )
/data1/sassie/satellite/seaice/sic_daily_v3/seaice_conc_daily_nh_f17_20120810_v03r01.nc ( 9

In [7]:
fnout = thisf.replace('seaice_conc_daily_nh_' , 'regridded/seaice_conc_daily_nh_regridded_')
fnout

'/data1/sassie/satellite/seaice/sic_daily_v3/regridded/seaice_conc_daily_nh_regridded_f17_20120801_v03r01.nc'